In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import math
from PIL import Image,ImageFilter
import pytesseract
import glob

In [2]:
img=cv2.imread("samples_marrow/Double sided20001.png")


In [3]:
def reduce_colors(img, n):
    Z = img.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = n
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res2 = res.reshape((img.shape))

    return res2 

In [7]:

crop_img = img[231:309, 96:1395]
cv2.imshow("cropped", crop_img)
cv2.waitKey(0)

255

In [89]:
im_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
#im_gray=cv2.equalizeHist(im_gray)
im_gray = cv2.cvtColor(reduce_colors(cv2.cvtColor(im_gray, cv2.COLOR_GRAY2BGR), 8), cv2.COLOR_BGR2GRAY)
ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
im_th=cv2.erode(im_th, kernel, iterations =2)
cv2.imshow("cropped", im_th)
cv2.waitKey(0)
img_bitwise=cv2.bitwise_not(im_th)
ctrs = cv2.findContours(img_bitwise,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

char_mask = np.zeros_like(im_th)
bounding_boxes = []

for contour in ctrs:
        x,y,w,h = cv2.boundingRect(contour)
        area = w * h
        center = (x + w/2, y + h/2)
        if (area > 550) and (area < 10000):
            x,y,w,h = x-4, y-4, w+8, h+8
            bounding_boxes.append((center, (x,y,w,h)))
            cv2.rectangle(char_mask,(x,y),(x+w,y+h),255,-1)
cv2.imshow("cropped", char_mask)
cv2.waitKey(0)           
clean = cv2.bitwise_not(cv2.bitwise_and(char_mask, char_mask, mask = img_bitwise))
bounding_boxes = sorted(bounding_boxes, key=lambda item: item[0][0])
characters = []
for center, bbox in bounding_boxes:
    x,y,w,h = bbox
    char_image = clean[y:y+h,x:x+w]
    characters.append((bbox, char_image))
    
output_img = cv2.cvtColor(clean, cv2.COLOR_GRAY2BGR)

i=0
for bbox, char_img in characters:
    to_crop=output_img[bbox[1]:(bbox[1] + bbox[3]),bbox[0]:(bbox[0] + bbox[2])]
    save_str='cropped_'+str(i)+'.png'
    save_dir=os.path.join('cropped_images/',save_str)
    cv2.imwrite(save_dir,to_crop)
    i=i+1
cv2.imshow("cropped", output_img)
cv2.waitKey(0)

255

In [97]:
i=0
for rect in characters:
    to_crop=crop_img[rect[1]:(rect[1] + rect[3]),rect[0]:(rect[0] + rect[2])]
    save_str='cropped_'+str(i)+'.png'
    save_dir=os.path.join('cropped_images/',save_str)
    cv2.imwrite(save_dir,to_crop)
    i=i+1
    #cv2.imwrite(save_dir,roi)
    # Draw the rectangles
    #cv2.rectangle(crop_img, (rect[0], rect[1]), (rect[0] + rect[2], rect[1] + rect[3]), (0, 255, 0), 1)

IndexError: tuple index out of range

In [69]:
cv2.imshow("Bounding boxes", crop_img)
cv2.waitKey(0)

255

In [185]:
num_images=25
i=0
for i in range(num_images):
    
    newImage = Image.new('L', (128, 128), (255)) #creates white canvas of 28x28 pixels
    for filename in glob.glob(os.path.join(sample_path, '*.png')):
    to_open_str='cropped_'+str(i)+'.png'
    open_dir=os.path.join('cropped_images/',to_open_str)
    im = Image.open(open_dir).convert('L')
    width = float(im.size[0])
    height = float(im.size[1])
    
    if width > height: #check which dimension is bigger
        #Width is bigger. Width becomes 20 pixels.
        nheight = int(round((50.0/width*height),0)) #resize height according to ratio width
        if (nheight == 0): #rare case but minimum is 1 pixel
            nheight = 1  
        # resize and sharpen
        img = im.resize((50,nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wtop = int(round(((128 - nheight)/2),0)) #caculate horizontal pozition
        newImage.paste(img, (40, wtop)) #paste resized image on white canvas
    else:
        #Height is bigger. Heigth becomes 20 pixels. 
        nwidth = int(round((50.0/height*width),0)) #resize width according to ratio height
        if (nwidth == 0): #rare case but minimum is 1 pixel
            nwidth = 1
         # resize and sharpen
        img = im.resize((nwidth,50), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
        wleft = int(round(((128 - nwidth)/2),0)) #caculate vertical pozition
        newImage.paste(img, (wleft, 40)) #paste resized image on white canvas
        
    to_save_str='test_'+str(i)+'.png'   
    save_dir=os.path.join('images_for_test/',to_save_str)
    newImage.save(save_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'cropped_images/cropped_1.png'

In [8]:
import glob
def image_preprocess_all(x_1,y_1,x_2,y_2,sample_path,save_cropped_path,save_test):
    image_ctr=0
    for filename in glob.glob(os.path.join(sample_path, '*.png')):
        img=cv2.imread(filename)
        crop_img = img[y_1:y_2, x_1:x_2]
        im_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
        #im_gray=cv2.equalizeHist(im_gray)
        im_gray = cv2.cvtColor(reduce_colors(cv2.cvtColor(im_gray, cv2.COLOR_GRAY2BGR), 8), cv2.COLOR_BGR2GRAY)
        ret, im_th = cv2.threshold(im_gray, 130, 255, cv2.THRESH_BINARY)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
        im_th=cv2.erode(im_th, kernel, iterations = 1)
        img_bitwise=cv2.bitwise_not(im_th)
        ctrs = cv2.findContours(img_bitwise,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

        char_mask = np.zeros_like(im_th)
        bounding_boxes = []

        for contour in ctrs:
                x,y,w,h = cv2.boundingRect(contour)
                area = w * h
                center = (x + w/2, y + h/2)
                if (area > 450) and (area < 10000):
                     x,y,w,h = x-4, y-4, w+8, h+8
                     bounding_boxes.append((center, (x,y,w,h)))
                     cv2.rectangle(char_mask,(x,y),(x+w,y+h),255,-1)

        clean = cv2.bitwise_not(cv2.bitwise_and(char_mask, char_mask, mask = img_bitwise))
        bounding_boxes = sorted(bounding_boxes, key=lambda item: item[0][0])
        characters = []
        for center, bbox in bounding_boxes:
            x,y,w,h = bbox
            char_image = clean[y:y+h,x:x+w]
            characters.append((bbox, char_image))

        output_img = cv2.cvtColor(clean, cv2.COLOR_GRAY2BGR)
        for bbox, char_img in characters:
            to_crop=output_img[bbox[1]:(bbox[1] + bbox[3]),bbox[0]:(bbox[0] + bbox[2])]
            save_str='cropped_'+str(image_ctr)+'.png'
            save_dir=os.path.join('cropped_images/',save_str)
            cv2.imwrite(save_dir,to_crop)
            image_ctr=image_ctr+1

        for i in range(image_ctr):
            newImage = Image.new('L', (128, 128), (255)) #creates white canvas of 28x28 pixels
            to_open_str='cropped_'+str(i)+'.png'
            open_dir=os.path.join(save_cropped_path,to_open_str)
            im = Image.open(open_dir).convert('L')
            width = float(im.size[0])
            height = float(im.size[1])

            if width > height: #check which dimension is bigger
                    #Width is bigger. Width becomes 20 pixels.
                nheight = int(round((50.0/width*height),0)) #resize height according to ratio width
                if (nheight == 0): #rare case but minimum is 1 pixel
                    nheight = 1  
                    # resize and sharpen
                img = im.resize((50,nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
                wtop = int(round(((128 - nheight)/2),0)) #caculate horizontal pozition
                newImage.paste(img, (40, wtop)) #paste resized image on white canvas
            else:
                #Height is bigger. Heigth becomes 20 pixels. 
                nwidth = int(round((50.0/height*width),0)) #resize width according to ratio height
                if (nwidth == 0): #rare case but minimum is 1 pixel
                    nwidth = 1
                     # resize and sharpen
                img = im.resize((nwidth,50), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
                wleft = int(round(((128 - nwidth)/2),0)) #caculate vertical pozition
                newImage.paste(img, (wleft, 40)) #paste resized image on white canvas

            to_save_str='test_'+str(i)+'.png'   
            save_dir=os.path.join(save_test,to_save_str)
            newImage.save(save_dir)

In [4]:
image_ctr=3207

In [7]:

def image_preprocess(x_1,y_1,x_2,y_2,sample_path,save_cropped_path,save_test,sample_image,min_area):
    global image_ctr
    #for filename in glob.glob(os.path.join(sample_path, '*.png')):
    filename=os.path.join(sample_path,sample_image)
    img=cv2.imread(filename)
    crop_img = img[y_1:y_2, x_1:x_2]
    im_gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)
    #im_gray=cv2.equalizeHist(im_gray)
    im_gray = cv2.cvtColor(reduce_colors(cv2.cvtColor(im_gray, cv2.COLOR_GRAY2BGR), 8), cv2.COLOR_BGR2GRAY)
    ret, im_th = cv2.threshold(im_gray, 230, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    im_th=cv2.erode(im_th, kernel, iterations = 1)
    img_bitwise=cv2.bitwise_not(im_th)
    ctrs = cv2.findContours(img_bitwise,cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]

    char_mask = np.zeros_like(im_th)
    bounding_boxes = []

    for contour in ctrs:
            x,y,w,h = cv2.boundingRect(contour)
            area = w * h
            center = (x + w/2, y + h/2)
            if (area > min_area) and (area < 10000):
                 x,y,w,h = x-2, y-2, w+2, h+2
                 bounding_boxes.append((center, (x,y,w,h)))
                 cv2.rectangle(char_mask,(x,y),(x+w,y+h),255,-1)
                    
    clean = cv2.bitwise_not(cv2.bitwise_and(char_mask, char_mask, mask = img_bitwise))
    bounding_boxes = sorted(bounding_boxes, key=lambda item: item[0][0])
    characters = []
    for center, bbox in bounding_boxes:
        x,y,w,h = bbox
        char_image = clean[y:y+h,x:x+w]
        characters.append((bbox, char_image))

    output_img = cv2.cvtColor(clean, cv2.COLOR_GRAY2BGR)
    for bbox, char_img in characters:
        to_crop=output_img[bbox[1]:(bbox[1] + bbox[3]),bbox[0]:(bbox[0] + bbox[2])]
        save_str='cropped_'+str(image_ctr)+'.png'
        save_dir=os.path.join('cropped_images/',save_str)
        cv2.imwrite(save_dir,to_crop)
        
        newImage = Image.new('L', (128, 128), (255)) #creates white canvas of 28x28 pixels
        to_open_str='cropped_'+str(image_ctr)+'.png'
        open_dir=os.path.join(save_cropped_path,to_open_str)
        im = Image.open(open_dir).convert('L')
        width = float(im.size[0])
        height = float(im.size[1])
        
        if width > height: #check which dimension is bigger
                #Width is bigger. Width becomes 20 pixels.
            nheight = int(round((50.0/width*height),0)) #resize height according to ratio width
            if (nheight == 0): #rare case but minimum is 1 pixel
                nheight = 1  
                # resize and sharpen
            img = im.resize((50,nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
            wtop = int(round(((128 - nheight)/2),0)) #caculate horizontal pozition
            newImage.paste(img, (40, wtop)) #paste resized image on white canvas
        else:
            #Height is bigger. Heigth becomes 20 pixels. 
            nwidth = int(round((50.0/height*width),0)) #resize width according to ratio height
            if (nwidth == 0): #rare case but minimum is 1 pixel
                nwidth = 1
                 # resize and sharpen
            img = im.resize((nwidth,50), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)
            wleft = int(round(((128 - nwidth)/2),0)) #caculate vertical pozition
            newImage.paste(img, (wleft, 40)) #paste resized image on white canvas
        
        to_save_str='test_'+str(image_ctr)+'.png'   
        save_dir=os.path.join(save_test,to_save_str)
        newImage.save(save_dir)
        image_ctr=image_ctr+1

In [6]:
sample_path='samples_marrow/'
save_cropped_path='cropped_images/'
save_test='images_for_test/'
sample_image='Double sided20001.png'

In [23]:

image_preprocess(x_1=96,y_1=231,x_2=1395,y_2=309,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [25]:

image_preprocess(x_1=940,y_1=354,x_2=2530,y_2=450,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [28]:
image_preprocess(x_1=640,y_1=358,x_2=692,y_2=420,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=250)

In [29]:
image_preprocess(x_1=110,y_1=482,x_2=640,y_2=560,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [30]:
image_preprocess(x_1=1350,y_1=486,x_2=2510,y_2=568,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [8]:
image_preprocess(x_1=372,y_1=725,x_2=584,y_2=800,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [9]:
image_preprocess(x_1=86,y_1=702,x_2=306,y_2=796,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [20]:
image_preprocess(x_1=886,y_1=708,x_2=1082,y_2=788,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [21]:
image_preprocess(x_1=110,y_1=840,x_2=2232,y_2=928,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [22]:
image_preprocess(x_1=104,y_1=966,x_2=1668,y_2=1054,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [27]:
image_preprocess(x_1=92,y_1=1176,x_2=374,y_2=1254,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [28]:
image_preprocess(x_1=430,y_1=1172,x_2=886,y_2=1266,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [29]:
image_preprocess(x_1=954,y_1=1174,x_2=1540,y_2=1250,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [30]:
image_preprocess(x_1=102,y_1=1300,x_2=824,y_2=1376,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [31]:
image_preprocess(x_1=878,y_1=1302,x_2=1772,y_2=1378,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [33]:
image_preprocess(x_1=106,y_1=1424,x_2=618,y_2=1498,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [ ]:
image_preprocess(x_1=106,y_1=1424,x_2=618,y_2=1498,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [34]:
image_preprocess(x_1=696,y_1=1430,x_2=1264,y_2=1500,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [35]:
image_preprocess(x_1=1526,y_1=1430,x_2=1994,y_2=1508,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=450)

In [46]:
image_preprocess(x_1=634,y_1=1910,x_2=694,y_2=1964,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=200)

In [7]:
sample_image='Double sided30001.png'

In [51]:
image_preprocess(x_1=2446,y_1=376,x_2=2528,y_2=452,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=1000)

In [67]:
image_preprocess(x_1=94,y_1=232,x_2=1410,y_2=324,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [68]:
sample_image='Double sided30003.png'

In [91]:
image_preprocess(x_1=948,y_1=357,x_2=1582,y_2=469,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [92]:
sample_image='Double sided30005.png'

In [111]:
image_preprocess(x_1=1843,y_1=1429,x_2=2121,y_2=1520,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [112]:
sample_image='Double sided30007.png'

In [127]:
image_preprocess(x_1=75,y_1=1629,x_2=363,y_2=1720,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [128]:
sample_image='Double sided30009.png'

In [141]:
image_preprocess(x_1=76,y_1=1426,x_2=2111,y_2=1505,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [142]:
sample_image='Double sided30013.png'

In [159]:
image_preprocess(x_1=1578,y_1=1418,x_2=2110,y_2=1512,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [160]:
sample_image='Double sided30013.png'

In [202]:
image_preprocess(x_1=4342,y_1=664,x_2=4983,y_2=749,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [203]:
sample_image='Double sided30015.png'

In [261]:
image_preprocess(x_1=69,y_1=355,x_2=667,y_2=446,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [262]:
sample_image='Double sided30017.png'

In [282]:
image_preprocess(x_1=1509,y_1=1890,x_2=2412,y_2=1977,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [283]:
sample_image='Double sided30019.png'

In [303]:
image_preprocess(x_1=1510,y_1=1875,x_2=2151,y_2=1966,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [304]:
sample_image='Double sided30021.png'

In [323]:
image_preprocess(x_1=1528,y_1=1892,x_2=2452,y_2=1976,sample_path=sample_path,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=300)

In [324]:
sample_image='Double sided30023.png'

In [342]:
image_preprocess(x_1=1506,y_1=1890,x_2=2126,y_2=1978,sample_path=sample_path,
                 save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                 min_area=300)

In [8]:
sample_image='Image_001.png'

In [77]:
image_preprocess(x_1=5316,y_1=376,x_2=6646,y_2=512,sample_path=sample_path
                 ,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=200)

In [55]:
sample_image='Image_003.png'

In [78]:
image_preprocess(x_1=5314,y_1=1500,x_2=6002,y_2=1652,sample_path=sample_path
                 ,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=1000)

In [79]:
sample_image='Image_005.png'

In [95]:
image_preprocess(x_1=5258,y_1=1544,x_2=6828,y_2=1610,sample_path=sample_path
                 ,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=1000)

In [97]:
sample_image='Image_009.png'

In [238]:
sample_path='samples/'
sample_image='SAMPLE10.png'

In [244]:
image_preprocess(x_1=258,y_1=1084,x_2=1134,y_2=1690,sample_path=sample_path
                 ,save_cropped_path=save_cropped_path,save_test=save_test,sample_image=sample_image,
                min_area=200)